#### Wednesday, October 30, 2024

[Build a Simple LLM Application with LCEL](https://python.langchain.com/docs/tutorials/llm_chain/)

mamba activate langchain

Ok. Nice! This works with LMStudio using hermes-3-llama-3.1-8b.

In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [ ]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [ ]:
from langchain_openai import ChatOpenAI

# model = ChatOpenAI(model="gpt-4")
# model = ChatOpenAI(model="gpt-3.5-turbo")

# model = ChatOpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio", temperature=0)
# use "qwen2.5-14b-instruct" being served by LM Studio on KAUWITB
model = ChatOpenAI(base_url="http://192.168.2.16:1234/v1", api_key="lm-studio", temperature=0)

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

In [3]:
model.invoke(messages)

AIMessage(content='Ciao!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 22, 'total_tokens': 25, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'qwen2.5-14b-instruct', 'system_fingerprint': 'qwen2.5-14b-instruct', 'finish_reason': 'stop', 'logprobs': None}, id='run-8fd129f8-2ca2-4aa7-8b60-bc803f3d00ed-0', usage_metadata={'input_tokens': 22, 'output_tokens': 3, 'total_tokens': 25, 'input_token_details': {}, 'output_token_details': {}})

#### OutputParsers

In [4]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [5]:
result = model.invoke(messages)

In [6]:
parser.invoke(result)

'Ciao!'

In [7]:
chain = model | parser

In [8]:
chain.invoke(messages)

'Ciao!'

#### Prompt Templates

(Do the next few cells call OpenAI or LMStudio?)

In [9]:
from langchain_core.prompts import ChatPromptTemplate

In [10]:
system_template = "Translate the following into {language}:"

In [11]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [12]:
result = prompt_template.invoke({"language": "italian", "text": "hi"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}), HumanMessage(content='hi', additional_kwargs={}, response_metadata={})])

In [13]:
result.to_messages()

[SystemMessage(content='Translate the following into italian:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='hi', additional_kwargs={}, response_metadata={})]

#### Chaining together components with LECL

In [14]:
chain = prompt_template | model | parser

In [15]:
chain.invoke({"language": "italian", "text": "hi"})

'ciao'

#### Serving with LangServe